# Preprocess EYATH Data

**!!! IMORTANT !!!**<br>
This script is meant to be used iteratively and help with the MANUAL cleaning of the Excel files and normalization to a common format. After this is done, the conversion to .csv files (ready to be analysed) is performed automatically.

The format of the final table (if all columns are present is the following) - May be modified in the future
| N   | E   | Time (Greek am) | Temp (°C) | Turbidity (NTU) | Chla (mg m⁻³) | Date       |
|-----|-----|----------------|-----------|-----------------|---------------|------------|
|     |     |                |           |                 |               |            |

EYATH In situ measurements so far have been given in Excel format following a similar structure.

<br> **Sampling method**:

- Samples collected from side of vessel (in the middle) at a 20cm distance
- Clean vessel
- Sampling at point with no sediment influence
- Turbidity measurements (might contain surface organic/inorganic particles transferred by the river or air but not the same for Chl-a measurements, as it is removed due to the methodology used
- Measurements from the first 10-15cm of the water column

<br> **Additional observations by EYATH for samples of 26/06/2025**:

- No waves
- No rain previous days
- No macrophytes or vegetation at the sampling stations
- High transparency in the water column
- Dominant algae species with low Chl-a content

<br>**Details on specific files**:
<br>
- `HELOISA_EYATH 2025-2026.xlsx` [Polyfytos Reservoir, 26/06/2025] -> `HELOISA_EYATH 2025-2026_cleaned.xlsx (the cleaned version)`<br>
*A collection of 30 samples were collected at surface level with wide spatial coverage and measuring Chl-a and turbidity levels. Water temperature was also measured.*
<br><br> **Excel formatting issues**:
    - N/E column names needed to be changed to latin
    - row 4 of E column and rows 3, 13, 14 and 18 in column Time (Greek am) had extra spaces
- `Data_Polyphytos_21-22.xlsx` [Polyfytos Reservoir, 04/10/2021, 20/01/2022, 17/06/2022, 30/09/2022] -> `Data_Polyphytos_21-22_cleaned.xlsx (the cleaned version)`<br>
*A collection of 30 samples per day (except for 20/01/2022) were collected at surface level with wide spatial coverage and measuring Chl-a and turbidity levels. Water temperature was also measured.*
<br><br> **Excel formatting issues**:
    - 20.01.2022 sheet did not follow the format of the rest, but no satellite overpass occured that day, so it was discarded
    - the 04.10.2021 was dropped as it did not coincide with satellite overpass (also it did not follow the general format) 
    - Temperature was only present in 17.06.2022. Was renamed to "Temp oC" to match the general format

<br>**Depth (m) column**:
The Depth (m) column is not retained for the final .csv, as it does not represent the sampling depth. If new information arises modifications will be performed accordingly.

Here you can choose the file you will preprocess

In [1]:
# Select the file here
file = "Data_Polyphytos_21-22_cleaned.xlsx" # "HELOISA_EYATH 2025-2026_cleaned.xlsx", "Data_Polyphytos_21-22_cleaned.xlsx"

insitu = "../data/xlsx/" + file

In [2]:
# Normalization to follow Degrees Decimal Minutes (DMM) format
def clean_coordinate(coord,direction):
    d,m,md = coord.split(" ")

    return str(int(d)) + " " + m + "." + md + " " + direction

Import necessary libraries

In [3]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

Some helper function that help with the validation of the Excel file format

In [4]:
# Validate column with regex
def validate_column(df: pd.DataFrame, col: str, pattern: str):
    # Check validity of column
    df["is_valid"] = df[col].astype(str).str.match(pattern)

    # Filter wrong rows
    invalid_rows = df[~df["is_valid"]].drop(columns="is_valid")

    # Overall pass/fail
    overall = "✅" if df["is_valid"].all() else "❌"

    print(f"Overall validation of column {col}:", overall)
    if not df["is_valid"].all():
        print("\nNon-conforming rows:\n", invalid_rows[col], "\n")
    
    return overall

# Check if all columns are present
def check_columns(df: pd.DataFrame, required: list[str], secondary_columns:bool=True) -> bool:
    missing = [col for col in required if col not in df.columns]
    if missing:
        if not secondary_columns:
            print("❌ There are missing columns. Either modify the file or the code based on your requirements.\n")
        else:
            print("⚠️ There are missing columns from the full list of expected ones. These columns are considered secondary, so you are juts given a warning.")
        print("Missing columns:", missing)
        return False, missing
    else:
        print("✅ All required columns are present.\n")
    return True, missing

### This cell helps the user to ensure the Excel follows the desired format (and correct it MANUALLY if needed until it conforms) and save it in a uniform csv format.

In [5]:
file = pd.ExcelFile(insitu)  

# For each sheet (=date)
for sheetName in file.sheet_names:
    print(f"Sheet Name: {sheetName}")
    # Read in situ data for the day
    df = pd.read_excel(insitu,sheet_name=sheetName,index_col=False)

    
    # Removing unnamed columns using drop function
    df.drop(df.columns[df.columns.str.contains(
        'unnamed', case=False)], axis=1, inplace=True)

    print(df.columns)

    # Validate sheet data
    val_dic = {"N": r"^\d{2,3} \d{2,3} \d{2,3}$",
               "E": r"^\d{2,3} \d{2,3} \d{2,3}$",
               "Time (Greek am)": r"^\d{1,2} \d{2}$",
               "Temp oC": r"^[+-]?\d+(\.\d+)?$",
               "Turbidity (NTU)": r"^[+-]?\d+(\.\d+)?$",
               "Chla (mg m-3)": r"^[+-]?\d+(\.\d+)?$",
               "Depth (m)": r"^[+-]?\d+(\.\d+)?$"}
    
    # Check if all columns are present
    excluded_keys = {'Temp oC', 'Depth (m)'}
    if not check_columns(df, val_dic.keys() - excluded_keys)[0]:
        exit
    _ , missing = check_columns(df, excluded_keys, secondary_columns=True)
    for m in missing:
        val_dic.pop(m, None)
    print(f"Keys after removing secondary missing keys: {val_dic.keys()}")

    # Validate format of columns
    atleast_one = True
    for c, pattern in val_dic.items():
        if not validate_column(df=df, col=c, pattern=pattern):
            atleast_one = False
    if not atleast_one:
        exit
    else:
        print("✅ ALL VALIDATION TESTS PASSED..........")
        df = df.drop(columns=['is_valid', 'Depth (m)'], errors='ignore')

    # Normalize excel to common csv format
    try:
        if(re.match(val_dic.get('N'), str(df['N'].iloc[0]))):
            # DMM
            print("Normalizing coordinates...")
            df['N']=df['N'].apply(lambda x:clean_coordinate(x,'N'))
            df['E']=df['E'].apply(lambda x:clean_coordinate(x,'E'))
        else:
            pass
        
        # Store date
        df['date']=pd.to_datetime(sheetName,dayfirst=True)
        # export to csv
        date_title =  datetime.strptime(sheetName.replace(" ", ""), "%d.%m.%Y").strftime("%Y%m%d")
        print("Saving converted .csv")
        df.to_csv(f'../data/csv/Data_Polyphytos_{date_title}.csv',index=False)
        
    except Exception as e:    
        print(e)
        print(f'Sheet Name : {sheetName} could not be processed')
        continue


Sheet Name: 17.06.2022
Index(['N', 'E', 'Time (Greek am)', 'Temp oC', 'Turbidity (NTU)',
       'Chla (mg m-3)', 'Depth (m)'],
      dtype='object')
✅ All required columns are present.

✅ All required columns are present.

Keys after removing secondary missing keys: dict_keys(['N', 'E', 'Time (Greek am)', 'Temp oC', 'Turbidity (NTU)', 'Chla (mg m-3)', 'Depth (m)'])
Overall validation of column N: ✅
Overall validation of column E: ✅
Overall validation of column Time (Greek am): ✅
Overall validation of column Temp oC: ✅
Overall validation of column Turbidity (NTU): ✅
Overall validation of column Chla (mg m-3): ✅
Overall validation of column Depth (m): ✅
✅ ALL VALIDATION TESTS PASSED..........
Normalizing coordinates...
Saving converted .csv
Sheet Name: 30.09.2022
Index(['N', 'E', 'Time (Greek am)', 'Turbidity (NTU)', 'Chla (mg m-3)',
       'Depth (m)'],
      dtype='object')
✅ All required columns are present.

⚠️ There are missing columns from the full list of expected ones. These colu